<a href="https://colab.research.google.com/github/Guhan2348519/dl-lab-tasks/blob/main/2348519_DL_lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def preprocess_images(directory, target_size=(224, 224), batch_size=32, augmentation=False):
    # ImageDataGenerator for loading and preprocessing images
    if augmentation:
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')
    else:
        datagen = ImageDataGenerator()

    # Flow images from directory in batches
    image_generator = datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical')  # Assuming classification task

    # Standardize pixel values
    images, labels = next(image_generator)
    standardized_images = images.astype('float32') / 255.0

    return standardized_images, labels


In [ ]:

import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to preprocess images
def preprocess_images(images, labels, target_size=(224, 224), batch_size=32, augmentation=False):
    # Standardize image sizes
    standardized_images = []
    for img in images:
        resized_img = tf.image.resize(img, target_size)
        standardized_images.append(resized_img)
    standardized_images = np.array(standardized_images)

    # Normalize pixel values
    standardized_images = standardized_images.astype('float32') / 255.0

In [ ]:
train_dir = 'C:\\Users\\USER\\OneDrive\\Desktop\\Datasets\\DL\\skinCancer - Lab6\\train'
test_dir = 'C:\\Users\\USER\\OneDrive\\Desktop\\Datasets\\DL\\skinCancer - Lab6\\test'

img_width, img_height = 224, 224

In [ ]:

import os
import cv2

def load_images(directory):
    images = []
    labels = []
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        for filename in os.listdir(label_dir):
            if filename.endswith(".jpg"):
                img = cv2.imread(os.path.join(label_dir, filename))
                img = cv2.resize(img, (img_width, img_height))
                images.append(img)
                if label == 'normal':
                    labels.append(0)  # 0 for normal (no tuberculosis)
                else:
                    labels.append(1)  # 1 for tuberculosis
    return images, labels

# Assuming 'train_dir' and 'test_dir' are the directories containing the training and test datasets
train_images, train_labels = load_images(train_dir)
test_images, test_labels = load_images(test_dir)

In [ ]:
def load_images(directory):
    images = []
    labels = []
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        for filename in os.listdir(label_dir):
            if filename.endswith(".jpg"):
                img = cv2.imread(os.path.join(label_dir, filename))
                img = cv2.resize(img, (img_width, img_height))
                images.append(img)
                if label == 'benign':
                    labels.append(0)
                else:
                    labels.append(1)
    return images, labelsA

train_images, train_labels = load_images(train_dir)
test_images, test_labels = load_images(test_dir)

In [ ]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
train_datagen.fit(train_images)